<a href="https://colab.research.google.com/github/rkaclfdl123/Dacon/blob/master/%EC%99%80%EC%9D%B8_%ED%92%88%EC%A7%88_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 메모리 변수 모두 제거
all = [var for var in globals() if var[0] != "_"]
for var in all:
    del globals()[var]

In [ ]:
import pandas as pd
wine1=pd.read_csv('/content/wine1_1.csv')

# from glob import glob
# glob('*')

In [ ]:
wine2=pd.read_csv('wine2.csv',sep='\t')


In [ ]:
wine1

,index,alcohol,class
0,7001,9.90,999.0
1,1,9.40,0.0
2,2,9.80,0.0
3,3,9.99,999.0
4,6498,9.80,0.0
...,...,...,...
6495,6493,11.20,1.0
6496,6494,9.60,1.0
6497,6495,9.40,1.0
6498,6496,12.80,1.0


In [ ]:
wine2

,index,sugar_PH
0,7000,2.3/3.5
1,1,1.9/3.51
2,2,2.6/3.2
3,6498,2.3/3.26
4,4,1.9/3.16
...,...,...
6493,6493,1.6/3.27
6494,6494,8/3.15
6495,6495,1.2/2.99
6496,6496,1.1/3.34


In [ ]:
intersection_idx = list(set(wine1['index']).intersection(wine2['index']))


# 교집합에 존재하는 데이터만 불러오기
wine = wine1.loc[
    wine1['index'].apply(lambda x: x in intersection_idx)].copy()

wine = wine.sort_values(by = 'index').reset_index(drop=True) 
wine['class'].min(), wine['class'].max()

print(wine.info())
#print(wine.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   index    6497 non-null   int64  
 1   alcohol  6497 non-null   float64
 2   class    6493 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 152.4 KB
None


In [ ]:
df=pd.merge(wine1,wine2)


In [ ]:
display(df.head())



,alcohol,class,sugar,PH
0,9.4,0.0,1.9,3.51
1,9.8,0.0,2.6,3.20
2,9.8,0.0,2.3,3.26
3,9.8,0.0,1.9,3.16
4,9.4,0.0,1.9,3.51


In [ ]:
# 필드분할, 파이썬 컴프리헨션(압축기법)
df['sugar']=[x.split('/')[0] if pd.notnull(x) else x for x in df['sugar_PH']]
df['PH']=[x.split('/')[1] if pd.notnull(x) else x for x in df['sugar_PH']]


In [ ]:
# 필드제거
df=df.drop(['index','sugar_PH'],axis=1)



In [ ]:
# 숫자자료인데 object로 되어 있는 자료 float64로 형 변환
col=['sugar','PH']
for x in col:
    df[x]=df[x].astype('float64')
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   class    6493 non-null   float64
 2   sugar    6497 non-null   float64
 3   PH       6497 non-null   float64
dtypes: float64(4)
memory usage: 253.8 KB


In [ ]:
# 방법3

dfTmp=pd.merge(wine1,wine2)
mainList=[]

for i in dfTmp['sugar_PH']:
  List=[]
  for j in i.split('/'):
    List.append(j)
  mainList.append(List)
print(mainList[:10])



import numpy as np
dfTmp['sugar']=np.array(mainList)[:,0]  # [:,0] 모든행, 0번열
dfTmp['PH']=np.array(mainList)[:,1]     # [:,1] 모든행, 1번열

[['1.9', '3.51'], ['2.6', '3.2'], ['2.3', '3.26'], ['1.9', '3.16'], ['1.9', '3.51'], ['1.8', '3.51'], ['1.6', '3.3'], ['1.2', '3.39'], ['2', '3.36'], ['6.1', '3.35']]


In [ ]:
col=['sugar','PH']
for x in col:
    df[x]=df[x].astype('float64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   class    6497 non-null   float64
 2   sugar    6497 non-null   float64
 3   PH       6497 non-null   float64
dtypes: float64(4)
memory usage: 253.8 KB


In [ ]:
# 또는 col의 속성값이 object인자료를 찾아서 변경하여 주세요.
for x in df.columns:
    if df[x].dtype=='object':
        df[x]=df[x].astype('float64')

In [ ]:
import time
start=time.time()

for x in df.columns:
    if df[x].dtype=='object':
        df[x]=df[x].astype('float64')

print(time.time()-start)

0.0002884864807128906


In [ ]:
import time
start=time.time()

[df[x].astype('float64') if df[x].dtype=='object' else x for x in df.columns]
[df[x].astype('float64') for x in df.columns  if df[x].dtype=='object']

print(time.time()-start)

0.0011055469512939453


In [ ]:
import time
start=time.time()

## 방법1 : 사용자가 변수를 직접 지정하고 일일이 명령어를 작업
df['sugar']=df['sugar'].astype('float64')
df['PH']=df['PH'].astype('float64')



## 방법2: 공통된 기능을 수행하는 변수명을 모아서 for로 반복함. 또는 사용자정의 함수 작성해도 됨
col=['sugar','PH']
for x in col:
    df[x]=df[x].astype('float64')
df.info()


## 방법3: 만약에 숫자로 되어 있는 object 관련 모든 변수를 숫자로 변형한다면 아래와 같이 작업
for x in df.columns:
    if df[x].dtype=='object':
        df[x]=df[x].astype('float64')

        
## 방법4: 방법3을 압축한 형식
[df[x].astype('float64') if df[x].dtype=='object' else x for x in df.columns]   # 방법4-1
[df[x].astype('float64') for x in df.columns  if df[x].dtype=='object']         # 방법4-2


# 방법5: 모두 바꾼다
df.astype('float64')

print(time.time()-start)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   class    6497 non-null   float64
 2   sugar    6497 non-null   float64
 3   PH       6497 non-null   float64
dtypes: float64(4)
memory usage: 253.8 KB
0.015384435653686523


In [ ]:
import time
start=time.time()

## class의 결측치를 0으로 치환하고자 함.
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

#  모든 숫자변수의 결측치를 0으로 채우기
df[df.select_dtypes(include=numerics).columns] =df[df.select_dtypes(include=numerics).columns].fillna(0)

print(time.time()-start)

0.004378557205200195


In [ ]:
# NUll값 채움
## class의 결측치를 0으로 치환하고자 함.
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']



In [ ]:
#  모든 숫자변수의 결측치를 0으로 채우기
df[df.select_dtypes(include=numerics).columns] =df[df.select_dtypes(include=numerics).columns].fillna(0)
df.isna().sum()


alcohol    0
class      0
sugar      0
PH         0
dtype: int64

In [ ]:
df.info()

X_data = df[['alcohol', 'sugar', 'PH']].to_numpy()
y_data = df['class'].to_numpy()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   class    6497 non-null   float64
 2   sugar    6497 non-null   float64
 3   PH       6497 non-null   float64
dtypes: float64(4)
memory usage: 253.8 KB


In [ ]:
X_data

array([[ 9.4 ,  1.9 ,  3.51],
       [ 9.8 ,  2.6 ,  3.2 ],
       [ 9.8 ,  2.3 ,  3.26],
       ...,
       [ 9.4 ,  1.2 ,  2.99],
       [12.8 ,  1.1 ,  3.34],
       [11.8 ,  0.8 ,  3.26]])

In [ ]:
y_data

array([0., 0., 0., ..., 1., 1., 1.])

In [ ]:
# [미션] x,y 데이터 분할
# class만 레이블(y값)으로 변환
# 그외는 특징값(X값임)

In [ ]:
# [미션] 데이터 분할
# train_x, test_X, train_y, test_y로 분할 30%
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score, roc_curve


train_x, test_x ,train_y, test_y = train_test_split(X_data, y_data, test_size=0.3,random_state=1)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)


(4547, 3) (1950, 3) (4547,) (1950,)


In [ ]:
## 참고 예측
# 새로운 new_X값도스케일링 하고 예측해야함.
# alcohol, sugar, PH값이

# new_X는 사용자가 웹에서 직접 입력할수도 있고, csv자료일수도 있고, 크롤링으로 얻어온 자료일수도 있고
# 또는 이미지 또는 사운드의 내용일수도 있음.

# 이 작업은 보통, 위에 있는 작업을 모두 피클링 또는 dump해서 객체화 시켜서 갖고 있게됨.

# 또는 머신러닝에서도 모델값만 갖고 있게 됨.
# 또는 딥러닝 같은경우는 모델값만 갖고 있게됨
import numpy as np
new_XX= np.array([
    [15, 30, 1.08]
])

from sklearn.linear_model import LogisticRegression # 만약 위의 내용이 없다면 꼭 해야함.
from sklearn.preprocessing import StandardScaler   # 만약 위의 내용이 없다면 꼭 해야함.
scaler = StandardScaler()  # 만약 위의 내용이 없다면 꼭 해야함.
scaler.fit(new_X)          # 만약 위의 내용이 없다면 꼭 해야함. 

newX_scaled = scaler.transform(new_XX)
y_hat = lr.predict(newX_scaled)

print(int(y_hat[0]))
#이 자료의 class가 1이면 화이트 화인/0이면 레드와인
classLabel=['red wine','wihte wine']
print(classLabel[int(y_hat[0])])

# 
확률=lr.predict_proba(new_XX)   # 출력물이 [0.,1.]  1이될확률이 1이예요. [0.2,0.8]은 1이될확률이 0.8이예요
classLabel[np.argmax(확률)]  # [0,1]은 1번째 값이 가장 큼. 즉 가장 큰값의 index가 나옴


NameError: ignored

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')


import matplotlib.pyplot as plt
sns.heatmap(cm, annot = True, fmt = 'd',cmap = 'Reds')
plt.xlabel('실제값')
plt.ylabel('예측값')
plt.xticks([0.5,1.5],['0(N)', '1(P)'])
plt.yticks([0.5,1.5],['0(N)', '1(P)'])
plt.show()
# cm
# 측값(P)'])


NameError: ignored

In [ ]:
## 재현율을 높여봅니다.
# predict_proba 분류결정 예측 확률을 반환

pred_proba = lr.predict_proba(test_scaled)
pred  = lr.predict(test_scaled)
print('pred_proba()결과 Shape : {0}'.format(pred_proba.shape))
print('pred_proba array에서 앞 3개만 샘플로 추출 \n:', pred_proba[:3])

# 예측 확률 array 와 예측 결과값 array 를 concatenate 하여 예측 확률과 결과값을 한눈에 확인
pred_proba_result = np.concatenate([pred_proba , pred.reshape(-1,1)],axis=1)
print('두개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n',pred_proba_result[:3])


In [ ]:
### 코드
def get_clf_eval(y_test , pred):
    confusion = confusion_matrix(y_test, pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    print('오차 행렬')
    print(confusion)
    print(' 정밀도: {0:.4f}, 재현율: {1:.4f}'.format(precision ,recall))
    
    # 리포트 출력
    #cm=confusion_matrix(test_y, lr.predict(test_scaled))
import confusion_matrix


In [ ]:
# 새로운 인덱스의 교집합

intersection_idx = list(set(wine1['index']).intersection(wine2['index']))


# 교집합에 존재하는 데이터만 불러오기
wine = wine1.loc[
    wine1['index'].apply(lambda x: x in intersection_idx)].copy()

wine = wine.sort_values(by = 'index').reset_index(drop=True) 
wine['class'].min(), wine['class'].max()

print(wine.info())
print(wine.head(20))
성
#######################################


## merge 로 두데이터 조인
df=pd.merge(wine, wine2)


# 파이썬 컴프리헨션(압축기법)
df['sugar']=[x.split('/')[0] if pd.notnull(x) else x for x in tmp['sugar_PH']]

df['sugar'] = df['sugar_PH'].apply(
    lambda x: x.split('/')[0] if pd.notnull(x) else x)

df['PH'] = df['sugar_PH'].apply(
    lambda x: x.split('/')[1] if pd.notnull(x) else x)

df.isna().sum()


KeyError: ignored